In [1]:
import pandas as pd
df=pd.read_csv('train.csv')
df=df.drop(columns=['image_link'])
df.sample(6)

,sample_id,catalog_content,price
48329,31710,Item Name: LEE KUM KEE SAUCE STIR FRY ORGNL 19...,18.750
33817,164860,Item Name: Spectrum Naturals Refined Canola Oi...,9.410
71961,135229,Item Name: ANAR GOURMET FOODS Tofu Tikka Masal...,11.990
64586,131331,Item Name: Lipton Beefy Onion Soup and Dip Mix...,1.990
35706,131571,"Item Name: Cameron's Coffee Vanilla Hazelnut, ...",7.570
2341,295799,Item Name: Prince of Peace Dong Quai & Red Dat...,6.115


In [2]:
import re
def clean_text(text):
    text=text.lower()
    text=re.sub(r"<.*?>"," ",text)
    text=re.sub(r'http\S+|www\S+|https\S+','',text)
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    text=re.sub(r'\s+',' ',text).strip()
    return text

In [3]:
df['cleaned_text']=df['catalog_content'].apply(clean_text)

In [4]:
df['cleaned_text'].str.len()

0          83
1         490
2         313
3        1264
4         139
         ... 
74995     748
74996    1645
74997     132
74998     869
74999     273
Name: cleaned_text, Length: 75000, dtype: int64

In [5]:
df['cleaned_text'].str.split().str.len()

0         18
1         80
2         59
3        205
4         26
        ... 
74995    130
74996    271
74997     21
74998    134
74999     47
Name: cleaned_text, Length: 75000, dtype: int64

In [6]:
df['cleaned_text'].str.len().describe()

count    75000.000000
mean       866.888160
std        816.969425
min         25.000000
25%        235.000000
50%        608.000000
75%       1223.000000
max       7715.000000
Name: cleaned_text, dtype: float64

In [7]:
from sentence_transformers import SentenceTransformer
model=SentenceTransformer('bert-base-nli-mean-tokens')
text=df['cleaned_text'].to_list()
embeddings=model.encode(text,batch_size=32,show_progress_bar=True,convert_to_numpy=True)

/Users/akshatsinha/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/akshatsinha/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 2344/2344 [12:53<00:00,  3.03it/s]


In [8]:
embeddings.shape

(75000, 768)

In [122]:
# Load the model (uses GPU automatically if available)
from sentence_transformers import SentenceTransformer
model=SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Convert cleaned text into embeddings
texts2=df["cleaned_text"].tolist()

embeddings2=model.encode(
    texts2,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)

print("Embedding shape:",embeddings2.shape)

/Users/akshatsinha/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 2344/2344 [34:34<00:00,  1.13it/s] 


Embedding shape: (75000, 768)


In [9]:
df["desc_len"]=df["cleaned_text"].str.len()
df["word_count"]=df["cleaned_text"].str.split().str.len()
df["num_digits"]=df["cleaned_text"].str.count(r"\d")

In [10]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
#pca=PCA(n_components=300,random_state=42)
#x_pca=pca.fit_transform(embeddings)
kmeans=KMeans(n_clusters=121,random_state=42)
df['automated_category_ID']=kmeans.fit_predict(embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [11]:
df['price_category_ID']=KMeans(n_clusters=5,random_state=42).fit_predict(df[['price']])

In [12]:
df['automated_category_ID'].value_counts()

automated_category_ID
65     1249
117    1156
17     1025
38      987
15      954
       ... 
73      296
120     290
68      266
1       250
100     231
Name: count, Length: 121, dtype: int64

In [14]:
df.where(df['automated_category_ID']==65).dropna()

,sample_id,catalog_content,price,cleaned_text,desc_len,word_count,num_digits,automated_category_ID,price_category_ID
41,95469.0,Item Name: Community Coffee Breakfast Blend 36...,12.23,item name community coffee breakfast blend 36 ...,1121.0,191.0,20.0,65.0,0.0
42,255445.0,Item Name: Don Francisco's 100% Colombia Supre...,6.99,item name don franciscos 100 colombia supremo ...,812.0,135.0,22.0,65.0,0.0
47,140715.0,"Item Name: Equator Coffees, Equator Blend Grou...",34.57,item name equator coffees equator blend ground...,1248.0,189.0,20.0,65.0,1.0
155,193140.0,Item Name: Perfect Samplers Extra Caffeine Ext...,39.99,item name perfect samplers extra caffeine extr...,1105.0,192.0,23.0,65.0,1.0
259,44704.0,Item Name: Starbucks VIA Decaf Italian Coffee ...,21.46,item name starbucks via decaf italian coffee 1...,800.0,139.0,18.0,65.0,0.0
...,...,...,...,...,...,...,...,...,...
74646,247437.0,Item Name: trücup Low-Acid Light Roast Coffee ...,29.95,item name trcup lowacid light roast coffee 2 l...,2423.0,400.0,12.0,65.0,1.0
74667,16814.0,"Item Name: Fresh Roasted Coffee, Organic Ethio...",59.97,item name fresh roasted coffee organic ethiopi...,1255.0,196.0,13.0,65.0,1.0
74669,297824.0,Item Name: Volcanica Butter Scotch Toffee Flav...,28.99,item name volcanica butter scotch toffee flavo...,1883.0,289.0,32.0,65.0,1.0
74766,200681.0,"Item Name: Starbucks K-Cup Coffee Pods, Brown ...",10.99,item name starbucks kcup coffee pods brown sug...,1216.0,196.0,19.0,65.0,0.0


In [16]:
df.where((df['automated_category_ID']==65) & (df['price_category_ID']==2)).dropna()

,sample_id,catalog_content,price,cleaned_text,desc_len,word_count,num_digits,automated_category_ID,price_category_ID
2451,181380.0,Item Name: Tuscany Oro Selecto Organic Espress...,77.96,item name tuscany oro selecto organic espresso...,979.0,151.0,7.0,65.0,2.0
2820,78241.0,Item Name: New England Coffee French Vanilla M...,81.93,item name new england coffee french vanilla me...,1260.0,203.0,25.0,65.0,2.0
3492,124158.0,"Item Name: Fresh Roasted Coffee, FRC Dark Roas...",129.99,item name fresh roasted coffee frc dark roast ...,2646.0,428.0,17.0,65.0,2.0
3677,242038.0,Item Name: The Bean Organic Coffee Company Wat...,81.18,item name the bean organic coffee company wate...,1873.0,291.0,21.0,65.0,2.0
4738,107391.0,Item Name: Community Coffee Breakfast Blend In...,110.95,item name community coffee breakfast blend ins...,1034.0,183.0,11.0,65.0,2.0
...,...,...,...,...,...,...,...,...,...
68235,57035.0,"Item Name: Fresh Roasted Coffee, Fair Trade Or...",74.99,item name fresh roasted coffee fair trade orga...,1339.0,214.0,14.0,65.0,2.0
70900,252836.0,"Item Name: Death Wish Coffee, Valhalla Java Da...",114.60,item name death wish coffee valhalla java dark...,3274.0,578.0,9.0,65.0,2.0
74042,197574.0,Item Name: 5lb Fair Trade Organic Certified Gu...,72.99,item name 5lb fair trade organic certified gua...,2467.0,420.0,26.0,65.0,2.0
74238,25502.0,Item Name: The Bean Organic Coffee Company Wat...,81.18,item name the bean organic coffee company wate...,1497.0,242.0,19.0,65.0,2.0


In [17]:
df['price_category_ID'].value_counts()

price_category_ID
0    53804
1    17098
2     3749
4      344
3        5
Name: count, dtype: int64

In [18]:
df.where(df['price_category_ID']==2).dropna()

,sample_id,catalog_content,price,cleaned_text,desc_len,word_count,num_digits,automated_category_ID,price_category_ID
7,222007.0,Item Name: VineCo Original Series Chilean Sauv...,94.00,item name vineco original series chilean sauvi...,2230.0,354.0,11.0,57.0,2.0
15,266475.0,Item Name: Organic Vinegar; Apple Cider\nValue...,81.44,item name organic vinegar apple cider value 10...,59.0,11.0,4.0,28.0,2.0
18,271423.0,Item Name: BulkSupplements.com Trehalose Powde...,109.97,item name bulksupplementscom trehalose powder ...,2757.0,433.0,18.0,51.0,2.0
20,267410.0,"Item Name: Food to Live Black-Eyed Peas, 25 Po...",98.99,item name food to live blackeyed peas 25 pound...,1604.0,281.0,16.0,98.0,2.0
38,196576.0,Item Name: Buchu Leaf (Organic) Tea (Loose) (4...,124.52,item name buchu leaf organic tea loose 4 oz zi...,577.0,101.0,32.0,34.0,2.0
...,...,...,...,...,...,...,...,...,...
74875,263110.0,"Item Name: Yocream Yogurt Mix, Cookies and Cre...",107.24,item name yocream yogurt mix cookies and cream...,104.0,20.0,5.0,0.0,2.0
74895,204173.0,Item Name: Tropical Mixed Dried Fruit Medley -...,89.99,item name tropical mixed dried fruit medley by...,2925.0,484.0,17.0,14.0,2.0
74926,247833.0,Item Name: Bloomingmore | Fresh Cut Gypsophili...,105.98,item name bloomingmore fresh cut gypsophilia 1...,1576.0,258.0,17.0,68.0,2.0
74959,52787.0,Item Name: Love and Quiches 1/4 Sheet Special ...,116.44,item name love and quiches 14 sheet special oc...,104.0,19.0,8.0,32.0,2.0


In [19]:
df['price'].where(df['price_category_ID']==3).dropna().describe()

count       5.000000
mean     1439.208000
std       771.543098
min       921.500000
25%      1010.540000
50%      1188.000000
75%      1280.000000
max      2796.000000
Name: price, dtype: float64

In [20]:
for i in range(5):
    print(f"Price Category ID: {i}")
    display(df['price'].where(df['price_category_ID']==i).dropna().describe())

Price Category ID: 0


count    53804.000000
mean        10.770182
std          6.575020
min          0.130000
25%          5.155000
50%          9.770000
75%         15.380000
max         25.910000
Name: price, dtype: float64

Price Category ID: 1


count    17098.000000
mean        41.077261
std         12.151528
min         25.920000
25%         30.582500
50%         37.990000
75%         49.708750
max         72.450000
Name: price, dtype: float64

Price Category ID: 2


count    3749.000000
mean      103.907431
std        27.801083
min        72.465000
25%        81.440000
50%        94.460000
75%       121.180000
max       189.900000
Name: price, dtype: float64

Price Category ID: 3


count       5.000000
mean     1439.208000
std       771.543098
min       921.500000
25%      1010.540000
50%      1188.000000
75%      1280.000000
max      2796.000000
Name: price, dtype: float64

Price Category ID: 4


count    344.000000
mean     276.195262
std       96.532720
min      189.980000
25%      212.912500
50%      239.165000
75%      311.675000
max      779.250000
Name: price, dtype: float64

In [21]:
tab_feats=df[['desc_len','word_count','num_digits','automated_category_ID','price_category_ID']].values

In [22]:
import numpy as np
x=np.hstack([embeddings,tab_feats])
y=df["price"].values
#x2=np.hstack([embeddings2,tab_feats])

In [23]:
#x=np.log1p(np.log1p(x))
y=np.log1p(np.log1p(y))
#x2=np.log1p(np.log1p(x2))

In [24]:
pd.DataFrame(x).describe()

,0,1,2,3,4,5,6,7,8,9,...,763,764,765,766,767,768,769,770,771,772
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,...,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,-0.768743,0.981656,-0.849260,0.075294,0.211644,-0.929654,0.227844,0.258686,0.518962,-0.341166,...,-0.890655,0.073654,-0.117545,0.356634,0.164734,866.888160,145.625160,14.505960,59.405613,0.346493
std,0.324655,0.306574,0.417551,0.189699,0.280356,0.355002,0.487686,0.375055,0.266908,0.304837,...,0.301120,0.207215,0.327481,0.408655,0.238027,816.969425,134.862567,8.531221,34.657198,0.617943
min,-1.808311,-0.331592,-2.439052,-0.921851,-0.890438,-2.107126,-1.403423,-1.589805,-0.896705,-1.428295,...,-2.045497,-0.920758,-1.830568,-1.400120,-0.872885,25.000000,6.000000,0.000000,0.000000,0.000000
25%,-0.995073,0.783355,-1.126740,-0.041104,0.008777,-1.174639,-0.114017,0.004275,0.365352,-0.551613,...,-1.105427,-0.056810,-0.330589,0.055013,0.009256,235.000000,41.000000,8.000000,29.000000,0.000000
50%,-0.801302,0.989269,-0.858102,0.081057,0.188742,-0.959499,0.216020,0.234608,0.545926,-0.343593,...,-0.897996,0.084015,-0.116298,0.340806,0.169028,608.000000,103.000000,13.000000,61.000000,0.000000
75%,-0.577487,1.191023,-0.583814,0.197945,0.392063,-0.723099,0.546405,0.487436,0.703474,-0.145283,...,-0.686805,0.213204,0.101598,0.639913,0.326693,1223.000000,204.000000,19.000000,88.000000,1.000000
max,0.841826,2.322398,1.278879,1.055276,1.470654,1.057010,2.262991,1.991799,1.442846,1.242883,...,0.538414,0.883568,1.315381,1.961597,1.056484,7715.000000,1330.000000,186.000000,120.000000,4.000000


In [59]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.3,random_state=42)

In [27]:
from xgboost import XGBRegressor
xgb=XGBRegressor(n_estimators=2500,
    learning_rate=0.02,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.2,
    reg_lambda=0.4,
    tree_method='hist',
    random_state=42)
xgb.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [60]:
y_xgb=xgb.predict(x_val)
np.expm1(np.expm1(y_xgb))

array([ 6.7866545, 47.425434 ,  7.391609 , ...,  6.3487067,  8.819453 ,
        6.6061373], dtype=float32)

In [61]:
np.expm1(np.expm1(y_val))

array([12.195, 38.54 , 17.86 , ...,  4.48 ,  5.79 ,  3.66 ])

In [62]:
def smape(y_true,y_pred):
    return np.mean(2*np.abs(y_true-y_pred)/(np.abs(y_true)+np.abs(y_pred)))*100

y_xgb=xgb.predict(x_val)
score=smape(np.expm1(np.expm1(y_val)),np.expm1(np.expm1(y_xgb)))
print("SMAPE:", score)

SMAPE: 40.37369206881055


In [31]:
testx=x[0:14380]
testy=y[0:14380]

In [32]:
predy=xgb.predict(testx)
smape(np.expm1(np.expm1(testy)),np.expm1(np.expm1(predy)))

17.474206173490582

In [33]:
np.expm1(np.expm1(testy))

array([ 4.89, 13.12,  1.97, ..., 18.95,  3.49, 19.99])

In [34]:
np.expm1(np.expm1(predy))

array([ 5.085075 , 12.56001  ,  1.9867314, ..., 13.092974 ,  4.176988 ,
       18.42844  ], dtype=float32)